In [226]:
import pandas as pd
import re
import underthesea
from string import punctuation
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [227]:
def read_data(path):
    doc = open(path, encoding='utf-8').read().split('\n')
    text = []
    label = []
    i = 1
    while i < len(doc):
        text.append(doc[i])
        label.append(doc[i + 1])
        i += 4

    df = pd.DataFrame({'review': text, 'sentiment': label})
    return df


In [228]:
def adding_label(data):
    label = data["sentiment"].values

    clean_label = []

    for i in range(len(label)):
        sample = label[i].split(",")
        for j in range(len(sample)):
            sample[j] = re.sub("[{}]", "", sample[j].strip())
        clean_label.append(sample)

    y = []
    for i in range(len(clean_label)):
        row = clean_label[i]

        j = 0
        while j < len(row):
            if(row[j+1] == 'positive'):
                y.append([i + 1 , row[j], 1])
            elif(row[j+1] == 'negative'):
                y.append([i + 1 , row[j], -1])
            else:
                y.append([i + 1 , row[j], 0])
            j += 2
    return y

In [229]:
df = read_data('Dataset_For_Work\\1-VLSP2018-SA-Hotel-train (7-3-2018).txt')

In [230]:
df.head()

,review,sentiment
0,Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...,"{HOTEL#DESIGN&FEATURES, positive}, {HOTEL#GENE..."
1,"Địa điểm thuận tiện, trong vòng bán kính 1,5km...","{LOCATION#GENERAL, positive}"
2,"Phục vụ, view đẹp, vị trí","{SERVICE#GENERAL, positive}, {HOTEL#GENERAL, p..."
3,"thuận tiện , sạch sẽ , vui vẻ hài lòng","{HOTEL#COMFORT, positive}, {HOTEL#CLEANLINESS,..."
4,Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...,"{LOCATION#GENERAL, positive}, {FACILITIES#GENE..."


In [231]:
all_classes = ["Text"]
all_classes += list(set([i[1] for i in adding_label(df)]))
y = adding_label(df)

In [232]:
all_classes

['Text',
 'HOTEL#CLEANLINESS',
 'LOCATION#GENERAL',
 'FOOD&DRINKS#STYLE&OPTIONS',
 'ROOM_AMENITIES#DESIGN&FEATURES',
 'HOTEL#MISCELLANEOUS',
 'FACILITIES#GENERAL',
 'SERVICE#GENERAL',
 'ROOM_AMENITIES#COMFORT',
 'FACILITIES#COMFORT',
 'ROOM_AMENITIES#GENERAL',
 'FOOD&DRINKS#PRICES',
 'ROOMS#PRICES',
 'FACILITIES#QUALITY',
 'ROOM_AMENITIES#MISCELLANEOUS',
 'HOTEL#GENERAL',
 'FACILITIES#MISCELLANEOUS',
 'HOTEL#COMFORT',
 'HOTEL#QUALITY',
 'ROOMS#COMFORT',
 'ROOM_AMENITIES#QUALITY',
 'HOTEL#PRICES',
 'ROOMS#QUALITY',
 'FOOD&DRINKS#QUALITY',
 'ROOMS#CLEANLINESS',
 'HOTEL#DESIGN&FEATURES',
 'ROOMS#GENERAL',
 'FACILITIES#DESIGN&FEATURES',
 'ROOMS#DESIGN&FEATURES',
 'FOOD&DRINKS#MISCELLANEOUS',
 'FACILITIES#CLEANLINESS',
 'ROOM_AMENITIES#CLEANLINESS',
 'FACILITIES#PRICES',
 'ROOMS#MISCELLANEOUS']

In [237]:
y

[[1, 'HOTEL#DESIGN&FEATURES', 1],
 [1, 'HOTEL#GENERAL', -1],
 [2, 'LOCATION#GENERAL', 1],
 [3, 'SERVICE#GENERAL', 1],
 [3, 'HOTEL#GENERAL', 1],
 [3, 'LOCATION#GENERAL', 1],
 [4, 'HOTEL#COMFORT', 1],
 [4, 'HOTEL#CLEANLINESS', 1],
 [4, 'SERVICE#GENERAL', 1],
 [5, 'LOCATION#GENERAL', 1],
 [5, 'FACILITIES#GENERAL', 1],
 [5, 'SERVICE#GENERAL', 1],
 [6, 'HOTEL#GENERAL', 1],
 [6, 'HOTEL#CLEANLINESS', 1],
 [6, 'SERVICE#GENERAL', 1],
 [7, 'ROOMS#CLEANLINESS', 1],
 [7, 'ROOM_AMENITIES#COMFORT', 1],
 [7, 'SERVICE#GENERAL', 1],
 [8, 'LOCATION#GENERAL', 1],
 [8, 'HOTEL#GENERAL', 1],
 [8, 'ROOMS#CLEANLINESS', 1],
 [8, 'ROOMS#COMFORT', 1],
 [8, 'FACILITIES#PRICES', -1],
 [9, 'HOTEL#COMFORT', 1],
 [10, 'ROOMS#CLEANLINESS', -1],
 [10, 'ROOM_AMENITIES#GENERAL', -1],
 [10, 'FOOD&DRINKS#STYLE&OPTIONS', -1],
 [10, 'SERVICE#GENERAL', 1],
 [11, 'HOTEL#GENERAL', 1],
 [11, 'SERVICE#GENERAL', 0],
 [12, 'ROOMS#PRICES', 1],
 [12, 'ROOMS#CLEANLINESS', 1],
 [12, 'ROOMS#DESIGN&FEATURES', 1],
 [12, 'ROOMS#GENERAL', 1

In [233]:
def data_with_sparse_matrix(df):
    all_classes = ["Text"]
    all_classes += list(set([i[1] for i in adding_label(df)]))
    y = adding_label(df)
    new_dat = pd.DataFrame(columns=all_classes)
    all_classes.remove("Text")
    new_dat.Text = df.review
    new_dat[all_classes] = 0
    sparse_matrix = np.zeros((len(df), len(all_classes)))
    dict_class = {}
    index = 0
    for i in all_classes:
        dict_class[i] = index
        index += 1
    sparse_matrix = list(sparse_matrix.astype(int))
    for i in y:
        sparse_matrix[i[0] - 1][dict_class[i[1]]] = i[2]
    for i in y:
        new_dat[i[1]][i[0] - 1] = i[2]
    final_dat = pd.DataFrame({"Text": new_dat.Text, "Label": sparse_matrix})

    data_with_all_classes = new_dat
    data_with_sparse = final_dat
    
    return data_with_sparse, data_with_all_classes

In [234]:
final_dat,new_dat = data_with_sparse_matrix(df)

In [235]:
final_dat.head() #Đây là cái để đem đi train

,Text,Label
0,Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1,..."
1,"Địa điểm thuận tiện, trong vòng bán kính 1,5km...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"Phục vụ, view đẹp, vị trí","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,"thuận tiện , sạch sẽ , vui vẻ hài lòng","[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...,"[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [236]:
new_dat.head() #Cái này làm thêm để nhìn cho đẹp

,Text,HOTEL#CLEANLINESS,LOCATION#GENERAL,FOOD&DRINKS#STYLE&OPTIONS,ROOM_AMENITIES#DESIGN&FEATURES,HOTEL#MISCELLANEOUS,FACILITIES#GENERAL,SERVICE#GENERAL,ROOM_AMENITIES#COMFORT,FACILITIES#COMFORT,...,ROOMS#CLEANLINESS,HOTEL#DESIGN&FEATURES,ROOMS#GENERAL,FACILITIES#DESIGN&FEATURES,ROOMS#DESIGN&FEATURES,FOOD&DRINKS#MISCELLANEOUS,FACILITIES#CLEANLINESS,ROOM_AMENITIES#CLEANLINESS,FACILITIES#PRICES,ROOMS#MISCELLANEOUS
0,Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,"Địa điểm thuận tiện, trong vòng bán kính 1,5km...",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Phục vụ, view đẹp, vị trí",0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"thuận tiện , sạch sẽ , vui vẻ hài lòng",1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
